In [1]:
import gmaps
import gmaps.datasets
import gmaps.geojson_geometries
import json
from matplotlib.cm import viridis, plasma
from matplotlib.colors import to_hex
import matplotlib.pyplot as plt
import pandas as pd
from config import census_key, g_key

# Configure gmaps
gmaps.configure(api_key=g_key)
ngeo = json.load(open("ncoordinates.json"))

#dict_keys(['type', 'crs', 'features'])
#print(len(ngeo['features']))
#print(ngeo.keys())
#print()
ngeo['features'][0]

{'type': 'Feature',
 'properties': {'PRI_NEIGH': 'Grand Boulevard',
  'SEC_NEIGH': 'BRONZEVILLE',
  'SHAPE_AREA': 48492503.1554,
  'SHAPE_LEN': 28196.837157},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-87.60670812560363, 41.81681377137387],
    [-87.60670480953505, 41.81657908583579],
    [-87.60670022648407, 41.816338713552454],
    [-87.60669581538588, 41.816099357727296],
    [-87.60668982110376, 41.8158118024656],
    [-87.60668357216157, 41.81556631526606],
    [-87.60667660553894, 41.815299912163404],
    [-87.6066796364493, 41.814994168113515],
    [-87.60668235893172, 41.81471953500853],
    [-87.60667153481008, 41.8142816453241],
    [-87.60666414094068, 41.81399460252956],
    [-87.60665643548599, 41.81366052091469],
    [-87.6066508943903, 41.81342058153228],
    [-87.60664694986733, 41.81317320016689],
    [-87.60664346744446, 41.81295477653956],
    [-87.60663536449937, 41.812655873932044],
    [-87.60662867423531, 41.81240396220561],
    [-87.60662516618756, 41

In [2]:
# Chicago Neighborhoods
data = pd.read_csv('../data/Neighborhood_Health.csv')
data_dict = data.filter(['Community Area Name', 'Assault (Homicide)'])
data_dict = data_dict.set_index('Community Area Name').to_dict()["Assault (Homicide)"]
rate_max = data[data["Assault (Homicide)"] == data["Assault (Homicide)"].max()]
rate_max_value = round(rate_max["Assault (Homicide)"],2).values[0]
rate_min = data[data["Assault (Homicide)"] == data["Assault (Homicide)"].min()]
rate_min_value = round(rate_min["Assault (Homicide)"],2).values[0]

In [3]:
#Scale the states values to lie between 0 and 1
min_nh = min(data_dict.values())
max_nh = max(data_dict.values())
nh_range = max_nh - min_nh

def calculate_color(neighborhood): #Convert the neighborhood value to a color
    normalized_nh = (neighborhood - min_nh) / nh_range # make neighborhood a number between 0 and 1
    inverse_nh = 1.0 - normalized_nh # invert neihborhood so that high value gives dark color
    mpl_color = plasma(inverse_nh) # transform the neighborhood value to a matplotlib color
    gmaps_color = to_hex(mpl_color, keep_alpha=False) # transform from a matplotlib color to a valid CSS color
    return gmaps_color

colors = []
for feature in ngeo['features']:
    geo_nh_name = feature['properties']['PRI_NEIGH']
    try:
        nh = data_dict[geo_nh_name]
        color = calculate_color(nh)
    except KeyError:
        # no value for that state: return default color
        color = (0, 0, 0, 0.3)
    colors.append(color)

In [4]:
fig = gmaps.figure()
nh_layer = gmaps.geojson_layer(
    ngeo,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.8)
fig.add_layer(nh_layer)
fig

Figure(layout=FigureLayout(height='420px'))